In [1]:
#Question 1

In [2]:
#1. Create Spark session as we did in the class. Make sure you enable Hive support

In [3]:
try:
    spark.stop()
except:
    pass
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Lecture_9").enableHiveSupport().getOrCreate()

In [4]:
#2. Load all files from "data\retail-data\by-day" into a single Spark DataFrame. There should be 300+ files in the folder.

In [5]:
#I moved the the by-day folder to the relative path of the notebook

In [6]:
df = spark.read.format("csv")\
.option("header", "true")\
.option("inferSchema", "true")\
.load("by-day/*.csv")
df.printSchema()
df.createOrReplaceTempView("dfTable")

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: timestamp (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: double (nullable = true)
 |-- Country: string (nullable = true)



In [7]:
#3. Print Schema of the DF created in 1.1

In [8]:
df.printSchema()

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: timestamp (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: double (nullable = true)
 |-- Country: string (nullable = true)



In [9]:
#4. Create SQL temp table to be used in Spark SQL for the current session. Name table "RetailData"

In [10]:
df.createOrReplaceTempView("RetailData")

In [11]:
#5. Display list of Hive Tables, permanent and temporary in the "defualt" schema

In [12]:
spark.sql("show tables").show()

+--------+----------+-----------+
|database| tableName|isTemporary|
+--------+----------+-----------+
| default| people_pq|      false|
| default| sample_07|      false|
| default| sample_08|      false|
|        |   dftable|       true|
|        |retaildata|       true|
+--------+----------+-----------+



In [13]:
#Question 2

In [14]:
#1. Cache the DF so that it wuill reside in memory after first "action"

In [15]:
df.cache()

DataFrame[InvoiceNo: string, StockCode: string, Description: string, Quantity: int, InvoiceDate: timestamp, UnitPrice: double, CustomerID: double, Country: string]

In [16]:
#2. Perform count() funnction on the DataFrame to cache the DF

In [17]:
df.count()

541909

In [18]:
#3. Use limit function to display first five records using both SQL and DF API. 
# Full content of each column should be displayed use proper parameter to show() function

In [19]:
spark.sql("select * from RetailData limit 5").show()

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|   580538|    23084|  RABBIT NIGHT LIGHT|      48|2011-12-05 08:38:00|     1.79|   14075.0|United Kingdom|
|   580538|    23077| DOUGHNUT LIP GLOSS |      20|2011-12-05 08:38:00|     1.25|   14075.0|United Kingdom|
|   580538|    22906|12 MESSAGE CARDS ...|      24|2011-12-05 08:38:00|     1.65|   14075.0|United Kingdom|
|   580538|    21914|BLUE HARMONICA IN...|      24|2011-12-05 08:38:00|     1.25|   14075.0|United Kingdom|
|   580538|    22467|   GUMBALL COAT RACK|       6|2011-12-05 08:38:00|     2.55|   14075.0|United Kingdom|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+



In [20]:
df.limit(5).show()

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|   580538|    23084|  RABBIT NIGHT LIGHT|      48|2011-12-05 08:38:00|     1.79|   14075.0|United Kingdom|
|   580538|    23077| DOUGHNUT LIP GLOSS |      20|2011-12-05 08:38:00|     1.25|   14075.0|United Kingdom|
|   580538|    22906|12 MESSAGE CARDS ...|      24|2011-12-05 08:38:00|     1.65|   14075.0|United Kingdom|
|   580538|    21914|BLUE HARMONICA IN...|      24|2011-12-05 08:38:00|     1.25|   14075.0|United Kingdom|
|   580538|    22467|   GUMBALL COAT RACK|       6|2011-12-05 08:38:00|     2.55|   14075.0|United Kingdom|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+



In [21]:
#Question 3

In [22]:
#Use Spark API and Spar SQL (two different commands) to find how many records are present in the DF loaded in "1"
#for every year-month combination. Sort by year-month in default sort order.

In [23]:
spark.sql("select count(*), DATE_FORMAT(InvoiceDate, 'Y MM') as Date from RetailData group by Date").show()

+--------+-------+
|count(1)|   Date|
+--------+-------+
|   84711|2011 11|
|   50226|2011 09|
|   39518|2011 07|
|   25525|2011 12|
|   35147|2011 01|
|   60742|2011 10|
|   35284|2011 08|
|   37030|2011 05|
|   42481|2010 12|
|   27707|2011 02|
|   36748|2011 03|
|   29916|2011 04|
|   36874|2011 06|
+--------+-------+



In [24]:
from pyspark.sql.functions import concat, year, month
df\
.withColumn("Year", year('InvoiceDate'))\
.withColumn("Month", month('InvoiceDate'))\
.withColumn("Date", concat("Year", "Month"))\
.groupBy("Date")\
.count()\
.sort('count', ascending=False)\
.show()

+------+-----+
|  Date|count|
+------+-----+
|201111|84711|
|201110|60742|
| 20119|50226|
|201012|42481|
| 20117|39518|
| 20115|37030|
| 20116|36874|
| 20113|36748|
| 20118|35284|
| 20111|35147|
| 20114|29916|
| 20112|27707|
|201112|25525|
+------+-----+



In [25]:
#Question 4

In [26]:
#1. How many records have Null in field "Description"?

In [27]:
from pyspark.sql.functions import instr,col
df.where(df["Description"].isNull()).count()

1454

In [28]:
#Double check with SQL
#spark.sql("select count(*) from RetailData where Description is null").show()

In [29]:
#2. Create new DF and populate field "Description" with 'NULL-STR' string

In [30]:
fill_col_vals = {"Description" : "NULL-STR"}
df2 = df.na.fill(fill_col_vals)

In [31]:
print(df2.where(df2["Description"].isNull()).count())
print(df2.where(df2["Description"] == "NULL-STR").count())

0
1454


In [32]:
#3. Create new Boolean column for each item in the following list: ['lunch','dinner','supper','breakfast']

In [33]:
from pyspark.sql.functions import when, lower
df3 = df2.withColumn("is_lunch", when(instr(lower(df2.Description), "lunch") > 0, True).otherwise(False))\
.withColumn("is_dinner", when(instr(lower(df2.Description), "dinner") > 0, True).otherwise(False))\
.withColumn("is_supper", when(instr(lower(df2.Description), "supper") > 0, True).otherwise(False))\
.withColumn("is_breakfast", when(instr(lower(df2.Description), "breakfast") > 0, True).otherwise(False))

In [34]:
#4. Populate the column as in the example below, and output first 50 records that had any of the items 
#in the item list matched to the content of the column "Description".

In [35]:
list_of_columns = ['is_lunch','is_dinner','is_supper','is_breakfast','Description']
df3.select(list_of_columns)\
.where((df3.is_lunch == True) | (df3.is_dinner == True) | (df3.is_supper == True) | (df3.is_breakfast == True)).show(50)

+--------+---------+---------+------------+--------------------+
|is_lunch|is_dinner|is_supper|is_breakfast|         Description|
+--------+---------+---------+------------+--------------------+
|   false|    false|    false|        true|FULL ENGLISH BREA...|
|    true|    false|    false|       false|LUNCH BAG VINTAGE...|
|    true|    false|    false|       false|LUNCH BAG PAISLEY...|
|    true|    false|    false|       false|LUNCH BAG SUKI DE...|
|    true|    false|    false|       false|CIRCUS PARADE LUN...|
|    true|    false|    false|       false|  LUNCH BAG WOODLAND|
|    true|    false|    false|       false| SPACEBOY LUNCH BOX |
|    true|    false|    false|       false|DOLLY GIRL LUNCH BOX|
|    true|    false|    false|       false|  LUNCH BAG WOODLAND|
|    true|    false|    false|       false| LUNCH BAG CARS BLUE|
|    true|    false|    false|       false|SKULL LUNCH BOX W...|
|    true|    false|    false|       false|STRAWBERRY LUNCH ...|
|    true|    false|    f

In [36]:
#2. Output frequencies for each Boolean column you have added in 4.1

In [37]:
df3\
.groupBy("is_lunch")\
.count()\
.sort('count')\
.show()

+--------+------+
|is_lunch| count|
+--------+------+
|    true| 18525|
|   false|523384|
+--------+------+



In [38]:
df3\
.groupBy("is_dinner")\
.count()\
.sort('count')\
.show()

+---------+------+
|is_dinner| count|
+---------+------+
|     true|  1746|
|    false|540163|
+---------+------+



In [39]:
df3\
.groupBy("is_supper")\
.count()\
.sort('count')\
.show()

+---------+------+
|is_supper| count|
+---------+------+
|    false|541909|
+---------+------+



In [40]:
df3\
.groupBy("is_breakfast")\
.count()\
.sort('count')\
.show()

+------------+------+
|is_breakfast| count|
+------------+------+
|        true|  1171|
|       false|540738|
+------------+------+

